In [2]:
%config Completer.use_jedi = False

In [6]:
import gym
from gym.wrappers import Monitor
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from collections import defaultdict ,namedtuple

In [7]:
from SK_replay_buffer import ReplayBuffer
from SK_agent import DQNAgent
from SK_model import DNN

In [8]:
env=Monitor(gym.make('LunarLander-v2'), './video', force=True)
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

State shape:  (8,)
Number of actions:  4


e:\anaconda\envs\drlnd\lib\site-packages\gym\wrappers\monitor.py:32: UserWarning: The Monitor wrapper is being deprecated in favor of gym.wrappers.RecordVideo and gym.wrappers.RecordEpisodeStatistics (see https://github.com/openai/gym/issues/2297)
  "The Monitor wrapper is being deprecated in favor of gym.wrappers.RecordVideo and gym.wrappers.RecordEpisodeStatistics (see https://github.com/openai/gym/issues/2297)"


#### Set up parameters

In [9]:
model=DNN(input_size=env.observation_space.shape[0], output_size=env.action_space.n,hidden_size1=64, hidden_size2=64)
rb=ReplayBuffer()

In [10]:
model.load_state_dict(torch.load('pred_model_weights.pth'))

<All keys matched successfully>

In [11]:
tgt_model=model=DNN(input_size=env.observation_space.shape[0], output_size=env.action_space.n,hidden_size1=64, hidden_size2=64)

In [12]:
agent=DQNAgent(env,rb,model,tgt_model,gamma=0.99,replace=500,epsilon=0) #Didn't think through train-test breakup properly so now have to add dummy values :)

#### Run model

In [13]:
np.random.seed(42)
rewards=0
for i in range(5):
    rewards=0
    state = env.reset()
    while True:
        action = agent.choose_action(env,state)
        state, reward, done, _ = env.step(action)
        rewards+=reward
        env.render()
        if done:
            break
env.close()